# Importing data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
import numpy as np
df = pd.read_csv('drive/MyDrive/Colab Notebooks/COOKIES_80.csv')
# df = pd.read_csv('C:\\Users\\Anonymous\\Downloads\\COOKIES_80.csv')
df.head()

,identifiant,device,id_visite,date_heure_visite,source,rubrique,chapitre,page,numero_page,temps_page,browser,os,region,pays
0,DI473484,Ordinateur,110488,29SEP16:09:50:50,Sites du Portail,NaN,NaN,NaN,0,0,Firefox 49.0,Windows 10,GR1315,GC8
1,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,NaN,NaN,NaN,0,0,Firefox 49.0,Windows 10,GR1320,GC8
2,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1029,1,58,Firefox 49.0,Windows 10,GR1320,GC8
3,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1025,4,35,Firefox 49.0,Windows 10,GR1320,GC8
4,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL12,PC140,PA1311,5,0,Firefox 49.0,Windows 10,GR1320,GC8


#COOKIES80 Cleaning

In [ ]:
#Replacing "Indetermines" and "-" with np.nan in order to replace it later

df["device"].replace("Indetermines", np.nan,inplace=True) 
df["source"].replace("-", np.nan,inplace=True) 

In [ ]:
#Concatinate different versions of "browser" in one unique version

df["browser"].replace(to_replace ='.*Firefox.*', value = 'Firefox', regex = True,inplace=True) 
df["browser"].replace(to_replace ='Chrom[a-z]+.*', value = 'Chrome', regex = True,inplace=True)  
df["browser"].replace(to_replace ='Samsung.*', value = 'Samsung', regex = True,inplace=True) 
df["browser"].replace(to_replace ='Safa[a-z]+.*', value = 'Safari', regex = True,inplace=True) 
df["browser"].replace(to_replace ='[a-zA-Z]+.*Safar[a-z]+.*', value = 'Safari', regex = True,inplace=True) 
df["browser"].replace(to_replace ='E[a-z]+.*', value = 'Edge', regex = True,inplace=True) 
df["browser"].replace(to_replace ='.*IE.*', value = 'Interet Explorer', regex = True,inplace=True) 
df["browser"].replace(to_replace ='Netscape.*', value = 'Netscape', regex = True,inplace=True)  
df.head()


,identifiant,device,id_visite,date_heure_visite,source,rubrique,chapitre,page,numero_page,temps_page,browser,os,region,pays
0,DI473484,Ordinateur,110488,29SEP16:09:50:50,Sites du Portail,NaN,NaN,NaN,0,0,Firefox,Windows 10,GR1315,GC8
1,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,NaN,NaN,NaN,0,0,Firefox,Windows 10,GR1320,GC8
2,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1029,1,58,Firefox,Windows 10,GR1320,GC8
3,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1025,4,35,Firefox,Windows 10,GR1320,GC8
4,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL12,PC140,PA1311,5,0,Firefox,Windows 10,GR1320,GC8


In [ ]:
len(df["browser"].unique())

24

In [ ]:
#Concatinate different versions of "os" in one unique version

df["os"].replace(to_replace ='.*Windows.*', value = 'Windows', regex = True,inplace=True)  
df["os"].replace(to_replace ='.*Android .*', value = 'Android', regex = True,inplace=True)  
df["os"].replace(to_replace ='.*iOS .*', value = 'iOS', regex = True,inplace=True)  
df["os"].replace(to_replace ='.*Mac .*', value = 'iOS', regex = True,inplace=True)  
df["os"].replace(to_replace ='OS X.*', value = 'iOS', regex = True,inplace=True)  

df.head()


,identifiant,device,id_visite,date_heure_visite,source,rubrique,chapitre,page,numero_page,temps_page,browser,os,region,pays
0,DI473484,Ordinateur,110488,29SEP16:09:50:50,Sites du Portail,NaN,NaN,NaN,0,0,Firefox,Windows,GR1315,GC8
1,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,NaN,NaN,NaN,0,0,Firefox,Windows,GR1320,GC8
2,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1029,1,58,Firefox,Windows,GR1320,GC8
3,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1025,4,35,Firefox,Windows,GR1320,GC8
4,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL12,PC140,PA1311,5,0,Firefox,Windows,GR1320,GC8


In [ ]:
len(df["os"].unique())

13

In [ ]:
#replace missing value with most frequent value

data = df.apply(lambda x: x.fillna(x.value_counts().index[0]))
data.head()

,identifiant,device,id_visite,date_heure_visite,source,rubrique,chapitre,page,numero_page,temps_page,browser,os,region,pays
0,DI473484,Ordinateur,110488,29SEP16:09:50:50,Sites du Portail,home_page,home_page,home_page,0,0,Firefox,Windows,GR1315,GC8
1,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,home_page,home_page,home_page,0,0,Firefox,Windows,GR1320,GC8
2,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1029,1,58,Firefox,Windows,GR1320,GC8
3,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL19,PC74,PA1025,4,35,Firefox,Windows,GR1320,GC8
4,DI473484,Ordinateur,255332,26SEP16:12:41:19,Moteurs,DL12,PC140,PA1311,5,0,Firefox,Windows,GR1320,GC8


In [ ]:
#combine diff rows with some user_id to the a unique row

data_grouped = data.groupby('identifiant').agg(lambda x : max(set(x.tolist()), key=x.tolist().count))

In [ ]:
data_grouped.head()

,device,id_visite,date_heure_visite,source,rubrique,chapitre,page,numero_page,temps_page,browser,os,region,pays
identifiant,,,,,,,,,,,,,
DI10,Ordinateur,278989,31AUG16:11:57:15,Moteurs,home_page,home_page,home_page,0,0,Chrome,iOS,GR474,GC1
DI1000005,Smartphone,624344,14SEP16:22:39:19,Acces Direct,DL15,PC147,PA577,1,0,Safari,iOS,GR463,GC1
DI1000026,Smartphone,372660,14SEP16:15:11:07,Autres visites RS,DL20,PC111,PA450,0,0,Safari,iOS,GR463,GC1
DI1000028,Smartphone,284126,14SEP16:12:24:54,Reseaux Sociaux,DL23,home_page,PA528,1,0,Chrome,Android,GR463,GC1
DI1000029,Ordinateur,161766,14SEP16:09:46:03,Acces Direct,DL15,PC140,PA9345,1,0,Chrome,Windows,GR463,GC1


In [ ]:
#sum of temps-page
grouped_single_temps_page = df.groupby('identifiant').agg({'temps_page':'sum' })

In [ ]:
data_m = pd.merge(data_grouped,grouped_single_temps_page, on="identifiant")
data_m.head()

,device,id_visite,date_heure_visite,source,rubrique,chapitre,page,numero_page,temps_page_x,browser,os,region,pays,temps_page_y
identifiant,,,,,,,,,,,,,,
DI10,Ordinateur,278989,31AUG16:11:57:15,Moteurs,home_page,home_page,home_page,0,0,Chrome,iOS,GR474,GC1,0
DI1000005,Smartphone,624344,14SEP16:22:39:19,Acces Direct,DL15,PC147,PA577,1,0,Safari,iOS,GR463,GC1,0
DI1000026,Smartphone,372660,14SEP16:15:11:07,Autres visites RS,DL20,PC111,PA450,0,0,Safari,iOS,GR463,GC1,0
DI1000028,Smartphone,284126,14SEP16:12:24:54,Reseaux Sociaux,DL23,home_page,PA528,1,0,Chrome,Android,GR463,GC1,10
DI1000029,Ordinateur,161766,14SEP16:09:46:03,Acces Direct,DL15,PC140,PA9345,1,0,Chrome,Windows,GR463,GC1,4510


In [ ]:
data_final = data_m.drop(['id_visite','date_heure_visite','temps_page_x','numero_page'],axis=1)

In [ ]:
data_final.head()

,device,source,rubrique,chapitre,page,browser,os,region,pays,temps_page_y
identifiant,,,,,,,,,,
DI10,Ordinateur,Moteurs,home_page,home_page,home_page,Chrome,iOS,GR474,GC1,0
DI1000005,Smartphone,Acces Direct,DL15,PC147,PA577,Safari,iOS,GR463,GC1,0
DI1000026,Smartphone,Autres visites RS,DL20,PC111,PA450,Safari,iOS,GR463,GC1,0
DI1000028,Smartphone,Reseaux Sociaux,DL23,home_page,PA528,Chrome,Android,GR463,GC1,10
DI1000029,Ordinateur,Acces Direct,DL15,PC140,PA9345,Chrome,Windows,GR463,GC1,4510


In [ ]:
X=data_final

X['identifiant'] = X.index
X.reset_index(drop=True)

#reading the target variables in order to have a column with all users ID and 0 or 1

Y = pd.read_csv ('drive/MyDrive/Colab Notebooks/SUBSCRIBERS_80.csv')
Y.head()
Liste_subscribers=Y["identifiant"].tolist()


#creating new dataframe to have all users

Liste_identifiant=X["identifiant"].unique()

Y_updated= pd.DataFrame(Liste_identifiant)
Y_updated.columns = ['identifiant']
Y_updated.head()

#affecting 1 or 0

Y_updated['Subscription'] = [1 if x in Liste_subscribers else 0 for x in Y_updated['identifiant']]
Y_updated.head()

# data_final = data_m.drop(['identifiant'],axis=1)
X.reset_index(drop=True,inplace=True)
X.head()


#merging the 2 data frames
final_XandY = pd.merge(Y_updated, X, how='inner', on = ['identifiant'])
print(final_XandY.shape)
final_XandY.head()


(298667, 12)


,identifiant,Subscription,device,source,rubrique,chapitre,page,browser,os,region,pays,temps_page_y
0,DI10,0,Ordinateur,Moteurs,home_page,home_page,home_page,Chrome,iOS,GR474,GC1,0
1,DI1000005,0,Smartphone,Acces Direct,DL15,PC147,PA577,Safari,iOS,GR463,GC1,0
2,DI1000026,0,Smartphone,Autres visites RS,DL20,PC111,PA450,Safari,iOS,GR463,GC1,0
3,DI1000028,0,Smartphone,Reseaux Sociaux,DL23,home_page,PA528,Chrome,Android,GR463,GC1,10
4,DI1000029,0,Ordinateur,Acces Direct,DL15,PC140,PA9345,Chrome,Windows,GR463,GC1,4510


In [ ]:
#converting cateogrial variables to dummy variables

dummy1=pd.get_dummies(final_XandY["device"])
dummy2=pd.get_dummies(final_XandY["source"])
dummy3=pd.get_dummies(final_XandY["rubrique"])
dummy4=pd.get_dummies(final_XandY["chapitre"])
dummy5=pd.get_dummies(final_XandY["browser"])
dummy6=pd.get_dummies(final_XandY["os"])
dummy7=pd.get_dummies(final_XandY["region"])
dummy8=pd.get_dummies(final_XandY["pays"])

print(dummy1.shape ,dummy2.shape ,dummy3.shape ,dummy4.shape )
print(dummy5.shape ,dummy6.shape ,dummy7.shape ,dummy8.shape )


(298667, 5) (298667, 12) (298667, 30) (298667, 164)
(298667, 24) (298667, 13) (298667, 896) (298667, 177)


In [ ]:
final_XandY=pd.concat([final_XandY,dummy1,dummy2,dummy3,dummy4,dummy5,dummy6,dummy7,dummy8],axis=1)
final_XandY.head()

,identifiant,Subscription,device,source,rubrique,chapitre,page,browser,os,region,pays,temps_page_y,Console,Ordinateur,Smartphone,TV connectee,Tablette,Acces Direct,Autres visites RS,Email marketing,Flux RSS,Liens Sponsorises,Moteurs,Notoriete,Publicite,Reseaux Sociaux,Sites affluents,Sites du Portail,Webmails,DL10,DL11,DL12,DL13,DL14,DL15,DL16,DL17,DL18,DL19,DL2,...,GC63,GC64,GC65,GC66,GC67,GC68,GC69,GC7,GC70,GC71,GC72,GC73,GC74,GC75,GC76,GC77,GC78,GC79,GC8,GC80,GC81,GC82,GC83,GC84,GC85,GC86,GC87,GC88,GC89,GC9,GC90,GC91,GC92,GC93,GC94,GC95,GC96,GC97,GC98,GC99
0,DI10,0,Ordinateur,Moteurs,home_page,home_page,home_page,Chrome,iOS,GR474,GC1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DI1000005,0,Smartphone,Acces Direct,DL15,PC147,PA577,Safari,iOS,GR463,GC1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DI1000026,0,Smartphone,Autres visites RS,DL20,PC111,PA450,Safari,iOS,GR463,GC1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,DI1000028,0,Smartphone,Reseaux Sociaux,DL23,home_page,PA528,Chrome,Android,GR463,GC1,10,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,DI1000029,0,Ordinateur,Acces Direct,DL15,PC140,PA9345,Chrome,Windows,GR463,GC1,4510,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
final_XandY.drop("device",axis=1,inplace=True)
final_XandY.drop("source",axis=1,inplace=True)
final_XandY.drop("rubrique",axis=1,inplace=True)
final_XandY.drop("chapitre",axis=1,inplace=True)
final_XandY.drop("page",axis=1,inplace=True)
final_XandY.drop("browser",axis=1,inplace=True)
final_XandY.drop("os",axis=1,inplace=True)
final_XandY.drop("region",axis=1,inplace=True)
final_XandY.drop("pays",axis=1,inplace=True)
final_XandY.head()

,identifiant,Subscription,temps_page_y,Console,Ordinateur,Smartphone,TV connectee,Tablette,Acces Direct,Autres visites RS,Email marketing,Flux RSS,Liens Sponsorises,Moteurs,Notoriete,Publicite,Reseaux Sociaux,Sites affluents,Sites du Portail,Webmails,DL10,DL11,DL12,DL13,DL14,DL15,DL16,DL17,DL18,DL19,DL2,DL20,DL21,DL22,DL23,DL24,DL25,DL28,DL29,DL3,...,GC63,GC64,GC65,GC66,GC67,GC68,GC69,GC7,GC70,GC71,GC72,GC73,GC74,GC75,GC76,GC77,GC78,GC79,GC8,GC80,GC81,GC82,GC83,GC84,GC85,GC86,GC87,GC88,GC89,GC9,GC90,GC91,GC92,GC93,GC94,GC95,GC96,GC97,GC98,GC99
0,DI10,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,DI1000005,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DI1000026,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,DI1000028,0,10,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,DI1000029,0,4510,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#( u can't change this cell position)
Y_entrainnement=final_XandY['Subscription'].values
Y_entrainnement.shape

(298667,)

In [ ]:
final_XandY.drop("Subscription",axis=1,inplace=True)
final_XandY.drop("identifiant",axis=1,inplace=True)
final_XandY.head()



,temps_page_y,Console,Ordinateur,Smartphone,TV connectee,Tablette,Acces Direct,Autres visites RS,Email marketing,Flux RSS,Liens Sponsorises,Moteurs,Notoriete,Publicite,Reseaux Sociaux,Sites affluents,Sites du Portail,Webmails,DL10,DL11,DL12,DL13,DL14,DL15,DL16,DL17,DL18,DL19,DL2,DL20,DL21,DL22,DL23,DL24,DL25,DL28,DL29,DL3,DL30,DL31,...,GC63,GC64,GC65,GC66,GC67,GC68,GC69,GC7,GC70,GC71,GC72,GC73,GC74,GC75,GC76,GC77,GC78,GC79,GC8,GC80,GC81,GC82,GC83,GC84,GC85,GC86,GC87,GC88,GC89,GC9,GC90,GC91,GC92,GC93,GC94,GC95,GC96,GC97,GC98,GC99
0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4510,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
## for test synchronisation


In [ ]:
##now we should concatinat column that arent commun between COOKIE80 and COOKIES20

# Cleaning COOKIES_20_final

In [ ]:
COOKIES_20 = pd.read_csv('drive/MyDrive/Colab Notebooks/COOKIES_20.csv')
COOKIES_20.head()

,identifiant,device,id_visite,date_heure_visite,source,rubrique,chapitre,page,numero_page,temps_page,browser,os,region,pays
0,DI1448416,Ordinateur,94672,13SEP16:08:52:09,Sites affluents,NaN,NaN,NaN,0,0,Firefox 48.0,Windows 10,GR482,GC1
1,DI1448416,Ordinateur,94672,13SEP16:08:52:09,Sites affluents,DL12,PC169,PA2170,2,0,Firefox 48.0,Windows 10,GR482,GC1
2,DI1448416,Ordinateur,94672,13SEP16:08:52:09,Sites affluents,DL5,PC188,PA2205,1,45,Firefox 48.0,Windows 10,GR482,GC1
3,DI144660,Indetermines,264958,28SEP16:14:29:26,-,NaN,NaN,NaN,0,0,Autres navigateurs,Autre OS,NaN,NaN
4,DI2197700,Ordinateur,274633,16SEP16:12:40:42,Moteurs,home_page,home_page,home_page,1,0,MsIE 11.x,Windows 7,GR463,GC1


In [ ]:
print(len(COOKIES_20["identifiant"].unique()))
print(COOKIES_20.shape)

74665
(503144, 14)


###Cleaning TEST_DATA:
same process was applied to the training data before in the first section

In [ ]:

COOKIES_20["device"].replace("Indetermines", np.nan,inplace=True) 
COOKIES_20["source"].replace("-", np.nan,inplace=True)


#Concatinate different versions of "browser" in one unique version

COOKIES_20["browser"].replace(to_replace ='.*Firefox.*', value = 'Firefox', regex = True,inplace=True) 
COOKIES_20["browser"].replace(to_replace ='Chrom[a-z]+.*', value = 'Chrome', regex = True,inplace=True)  
COOKIES_20["browser"].replace(to_replace ='Samsung.*', value = 'Samsung', regex = True,inplace=True) 
COOKIES_20["browser"].replace(to_replace ='Safa[a-z]+.*', value = 'Safari', regex = True,inplace=True) 
COOKIES_20["browser"].replace(to_replace ='[a-zA-Z]+.*Safar[a-z]+.*', value = 'Safari', regex = True,inplace=True) 
COOKIES_20["browser"].replace(to_replace ='E[a-z]+.*', value = 'Edge', regex = True,inplace=True) 
COOKIES_20["browser"].replace(to_replace ='.*IE.*', value = 'Interet Explorer', regex = True,inplace=True) 
COOKIES_20["browser"].replace(to_replace ='Netscape.*', value = 'Netscape', regex = True,inplace=True)  
COOKIES_20.head()


#Concatinate different versions of "os" in one unique version

COOKIES_20["os"].replace(to_replace ='.*Windows.*', value = 'Windows', regex = True,inplace=True)  
COOKIES_20["os"].replace(to_replace ='.*Android .*', value = 'Android', regex = True,inplace=True)  
COOKIES_20["os"].replace(to_replace ='.*iOS .*', value = 'iOS', regex = True,inplace=True)  
COOKIES_20["os"].replace(to_replace ='.*Mac .*', value = 'iOS', regex = True,inplace=True)  
COOKIES_20["os"].replace(to_replace ='OS X.*', value = 'iOS', regex = True,inplace=True)  
COOKIES_20.head()


#replace missing value with most frequent value

COOKIES_20 = COOKIES_20.apply(lambda x: x.fillna(x.value_counts().index[0]))
COOKIES_20.head()


#combine diff rows with some user_id to the a unique row

data_grouped = COOKIES_20.groupby('identifiant').agg(lambda x : max(set(x.tolist()), key=x.tolist().count))


#sum of temps-page
grouped_single_temps_page = COOKIES_20.groupby('identifiant').agg({'temps_page':'sum' })



COOKIES_20_final = pd.merge(data_grouped,grouped_single_temps_page, on="identifiant")
COOKIES_20_final.head()


COOKIES_20_final = COOKIES_20_final.drop(['id_visite','date_heure_visite','temps_page_x','numero_page'],axis=1)

COOKIES_20_final.head()

,device,source,rubrique,chapitre,page,browser,os,region,pays,temps_page_y
identifiant,,,,,,,,,,
DI100001,Ordinateur,Acces Direct,home_page,home_page,home_page,Autres navigateurs,Autre OS,GR463,GC1,0
DI1000138,Ordinateur,Acces Direct,home_page,home_page,home_page,Interet Explorer,Windows,GR463,GC1,0
DI1000166,Ordinateur,Moteurs,home_page,PC68,home_page,Interet Explorer,Windows,GR463,GC1,0
DI1000167,Smartphone,Acces Direct,home_page,home_page,home_page,Samsung,Android,GR463,GC1,1722
DI1000188,Ordinateur,Reseaux Sociaux,DL15,PC147,PA577,Firefox,Windows,GR463,GC1,0


In [ ]:
List_id_cookies20=COOKIES_20_final["identifiant"].values

In [ ]:

#converting cateogrial variables to dummy variables

dummy1=pd.get_dummies(COOKIES_20_final["device"])
dummy2=pd.get_dummies(COOKIES_20_final["source"])
dummy3=pd.get_dummies(COOKIES_20_final["rubrique"])
dummy4=pd.get_dummies(COOKIES_20_final["chapitre"])
dummy5=pd.get_dummies(COOKIES_20_final["browser"])
dummy6=pd.get_dummies(COOKIES_20_final["os"])
dummy7=pd.get_dummies(COOKIES_20_final["region"])
dummy8=pd.get_dummies(COOKIES_20_final["pays"])

# dummy8.head()
# print(dummy1.shape ,dummy2.shape ,dummy3.shape ,dummy4.shape )
# print(dummy5.shape ,dummy6.shape ,dummy7.shape ,dummy8.shape )


COOKIES_20_final=pd.concat([COOKIES_20_final,dummy1,dummy2,dummy3,dummy4,dummy5,dummy6,dummy7,dummy8],axis=1)



COOKIES_20_final.drop("device",axis=1,inplace=True)
COOKIES_20_final.drop("source",axis=1,inplace=True)
COOKIES_20_final.drop("rubrique",axis=1,inplace=True)
COOKIES_20_final.drop("chapitre",axis=1,inplace=True)
COOKIES_20_final.drop("page",axis=1,inplace=True)
COOKIES_20_final.drop("browser",axis=1,inplace=True)
COOKIES_20_final.drop("os",axis=1,inplace=True)
COOKIES_20_final.drop("region",axis=1,inplace=True)
COOKIES_20_final.drop("pays",axis=1,inplace=True)
COOKIES_20_final.head()


print(COOKIES_20_final.shape)

# COOKIES_20_final.drop("identifiant",axis=1,inplace=True)
COOKIES_20_final.head()




# COOKIES_20_final=COOKIES_20_final.values
# COOKIES_20_final


(74665, 960)


,temps_page_y,Console,Ordinateur,Smartphone,Tablette,Acces Direct,Autres visites RS,Email marketing,Flux RSS,Liens Sponsorises,Moteurs,Notoriete,Publicite,Reseaux Sociaux,Sites affluents,Sites du Portail,Webmails,DL10,DL11,DL12,DL13,DL15,DL16,DL17,DL18,DL19,DL20,DL21,DL22,DL23,DL24,DL25,DL28,DL29,DL3,DL30,DL31,DL4,DL5,DL6,...,GC60,GC61,GC62,GC63,GC64,GC65,GC67,GC68,GC69,GC7,GC70,GC71,GC72,GC73,GC74,GC76,GC77,GC78,GC79,GC8,GC80,GC81,GC82,GC83,GC84,GC85,GC86,GC87,GC88,GC89,GC9,GC90,GC91,GC93,GC94,GC95,GC96,GC97,GC98,GC99
identifiant,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DI100001,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000138,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000166,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000167,1722,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000188,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(dummy1.shape ,dummy2.shape ,dummy3.shape ,dummy4.shape )
print(dummy5.shape ,dummy6.shape ,dummy7.shape ,dummy8.shape )

(74665, 4) (74665, 12) (74665, 27) (74665, 148)
(74665, 19) (74665, 11) (74665, 591) (74665, 147)


# Preparing data for ML model

### Determining non commun variable

In [ ]:
List_variables_Test=COOKIES_20_final.columns
len(List_variables_Test)

960

In [ ]:
List_variables_Train=final_XandY.columns
len(List_variables_Train)

1322

In [ ]:
TEST_not_in_train=np.setdiff1d(List_variables_Test,List_variables_Train)
print("this number of var dosent exist in TRAIN ",len(TEST_not_in_train))
# yields the elements in `list_2` that are NOT in `list_1`

this number of var dosent exist in TRAIN  77


In [ ]:
TRAIN_not_in_test=np.setdiff1d(List_variables_Train,List_variables_Test)
print("this number of var dosent exist in TRAIN ",len(TRAIN_not_in_test))
# yields the elements in `list_2` that are NOT in `list_1`

this number of var dosent exist in TRAIN  439


### Adding non existing column to each dataset
the number of variable will be 1391

In [ ]:
#### adding column in TEST data
for i in TRAIN_not_in_test:
  COOKIES_20_final[i]=0
COOKIES_20_final.head()


,temps_page_y,Console,Ordinateur,Smartphone,Tablette,Acces Direct,Autres visites RS,Email marketing,Flux RSS,Liens Sponsorises,Moteurs,Notoriete,Publicite,Reseaux Sociaux,Sites affluents,Sites du Portail,Webmails,DL10,DL11,DL12,DL13,DL15,DL16,DL17,DL18,DL19,DL20,DL21,DL22,DL23,DL24,DL25,DL28,DL29,DL3,DL30,DL31,DL4,DL5,DL6,...,GR96,GR960,GR962,GR967,GR969,GR97,GR974,GR979,GR98,GR980,GR994,GR996,Navigateur Nokia,Nintendo DS,PC117,PC125,PC150,PC176,PC182,PC193,PC207,PC26,PC28,PC42,PC45,PC48,PC5,PC55,PC57,PC61,PC7,PC75,PC80,PC82,PC85,PC98,Symbian OS,TV connectee,Teleca-Obigo,Wii
identifiant,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DI100001,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000138,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000166,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000167,1722,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
DI1000188,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#### adding column in TRAIN data
for i in TEST_not_in_train:
  final_XandY[i]=0
final_XandY.head()

,temps_page_y,Console,Ordinateur,Smartphone,TV connectee,Tablette,Acces Direct,Autres visites RS,Email marketing,Flux RSS,Liens Sponsorises,Moteurs,Notoriete,Publicite,Reseaux Sociaux,Sites affluents,Sites du Portail,Webmails,DL10,DL11,DL12,DL13,DL14,DL15,DL16,DL17,DL18,DL19,DL2,DL20,DL21,DL22,DL23,DL24,DL25,DL28,DL29,DL3,DL30,DL31,...,GR270,GR271,GR354,GR379,GR434,GR508,GR517,GR521,GR522,GR552,GR556,GR577,GR583,GR585,GR602,GR609,GR617,GR639,GR641,GR659,GR696,GR766,GR795,GR831,GR848,GR853,GR902,GR908,GR917,GR922,GR929,GR936,GR955,PC120,PC144,PC17,PC195,PC30,PC4,Unix
0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4510,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Converting df into arrays

In [ ]:
X_entrainnement=final_XandY.values
X_entrainnement.shape

(298667, 1399)

In [ ]:
Y_entrainnement.shape

(298667,)

In [ ]:
COOKIES_20_final=COOKIES_20_final.values
COOKIES_20_final.shape

(74665, 1399)

# Machine Learning

###Example of logistic Reg

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
#splitting data

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(X_entrainnement, Y_entrainnement, test_size = 0.4,random_state=42)

In [ ]:
print(x_training_data.shape, x_test_data.shape, y_training_data.shape, y_test_data.shape)

(179200, 1399) (119467, 1399) (179200,) (119467,)


In [ ]:
#Training our model

LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_training_data,y_training_data)

In [ ]:
#Predecting Y_test

yhat_proba = LR.predict_proba(x_test_data)
yhat = LR.predict(x_test_data)

In [ ]:
#measuring the accuracy

from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import log_loss

In [ ]:
jaccard_similarity_score(y_test_data, yhat)

In [ ]:
print ("LogLoss: : %.2f" % log_loss(y_test_data, yhat_proba))


# Predecting 

In [ ]:
List_id_cookies20

In [ ]:
yhat = LR.predict(COOKIES_20_final)

In [ ]:
result = pd.DataFrame(yhat, index =List,columns =['Y']) 
df.to_csv(r'Warriors_Subscribers_20.csv.csv')